In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from datetime import datetime

In [81]:
url = 'https://www.espn.com/golf/leaderboard'
headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
response = requests.get(url, headers=headers)
# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the HTML content of the webpage
    soup = BeautifulSoup(response.content, "html.parser")
    
    # Find the table with the specified class
    table = soup.find("tbody", class_="Table__TBODY")
    
    # Check if the table was found
    
    # Extract the rows of the table
    rows = table.find_all("tr")
    
    # Initialize an empty list to store the table data
    data = []
    
    # Loop through each row and extract the data
    for row in rows:
        # Extract the cells (td) of the row
        cells = row.find_all("td")
        
        # Extract the text content of each cell and append to the data list
        row_data = [cell.get_text() for cell in cells]
        data.append(row_data)
    headers = soup.find_all('th')
    header_texts = [header.text.strip() for header in headers]
    # Convert the data list into a pandas DataFrame
    df = pd.DataFrame(data)
    df = df[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]]
    df.columns = [x for x in header_texts]
    
    df['R3'] = df['R3'].replace('--', '0')
    df['R4'] = df['R4'].replace('--', '0')

    # Filter DataFrame
    df = df.dropna(subset = ['PLAYER'])
    filtered_df = df[~df['PLAYER'].str.endswith('(a)')]
    filtered_df = filtered_df[filtered_df['SCORE'] != 'CUT']
    filtered_df["TODAY"] = filtered_df["TODAY"].replace({
            'E': '0', 
        })
    filtered_df["R3"] = filtered_df["R3"].replace({
            'E': '0', 
        })

    for idx, row in filtered_df.iterrows(): 
            filtered_df.at[idx, 'TODAY'] = int(str(row['TODAY']).strip('+'))
            filtered_df.at[idx, 'R3'] = int(str(row['R3']).strip('+'))
            
    max_today = filtered_df['TODAY'].max() 
    max_r3 = filtered_df['R3'].max()

    # If we are on Saturday (R3) only add R1 and R2 else add R1 R2 and R3 
    current_date = datetime.now().date().strftime('%d-%m-%Y')

    for idx, row in df.iterrows(): 
        if row['SCORE'] == 'CUT': 
            if current_date == '12-04-2024': 
                df.at[idx, 'SCORE'] = (int(row['R1']) - 72) + (int(row['R2']) - 72) 
            elif current_date == '13-04-2024': 
                df.at[idx, 'SCORE'] = (int(row['R1']) - 72) + (int(row['R2']) - 72) + max_today
            elif current_date == '14-04-2024': 
                df.at[idx, 'SCORE'] = (int(row['R1']) - 72) + (int(row['R2']) - 72) + (max_r3 - 72) + max_today
    
    
    df = df[['PLAYER', 'SCORE']].rename(columns = {'PLAYER': 'golfer_name', 'SCORE': 'score'})
    
    df["golfer_name"] = df["golfer_name"].replace({
            'Ludvig Åberg': 'Ludvig Aberg',
            'Byeong Hun An': 'Byeong-Hun An', 
            'Nicolai Højgaard': 'Nicolai Hojgaard',
            'Joaquín Niemann': 'Joaquin Niemann', 
            'Christo Lamprecht (a)': 'Christo Lamprecht', 
            'Jasper Stubbs (a)': 'Jasper Stubbs',
            'Neal Shipley (a)': 'Neal Shipley', 
            'Santiago de la Fuente (a)': 'Santiago De la Fuente', 
            'Stewart Hagestad (a)': 'Stewart Hagestad', 
            'Thorbjørn Olesen': 'Thorbjorn Olesen'
        })
        
    df["score"] = df["score"].replace({
        'E': '0', 
    })
    df.columns = df.columns.get_level_values(0)
    df = df.dropna()
    for idx, row in df.iterrows(): 
        df.at[idx, 'score'] = int(str(row['score']).strip('+'))

In [82]:
df

,golfer_name,score
0,Max Homa,-6
1,Scottie Scheffler,-6
2,Bryson DeChambeau,-6
3,Nicolai Hojgaard,-4
4,Collin Morikawa,-3
...,...,...
85,Dustin Johnson,13
86,Gary Woodland,13
87,Adrian Meronk,14
88,Nick Taylor,14


In [66]:
# df['R3'] = df['R3'].replace('--', 0)
# df['R4'] = df['R4'].replace('--', 0)

# # Filter DataFrame
# df = df.dropna(subset = ['PLAYER'])
# filtered_df = df[~df['PLAYER'].str.endswith('(a)')]
# filtered_df = filtered_df[filtered_df['SCORE'] != 'CUT']
# filtered_df["TODAY"] = filtered_df["TODAY"].replace({
#         'E': '0', 
#     })

# for idx, row in filtered_df.iterrows(): 
#         filtered_df.at[idx, 'TODAY'] = int(str(row['TODAY']).strip('+'))
        
# max_today = filtered_df['TODAY'].max() 

# # If we are on Saturday (R3) only add R1 and R2 else add R1 R2 and R3 
# current_date = datetime.now().date().strftime('%d-%m-%Y')

# for idx, row in df.iterrows(): 
#     if row['SCORE'] == 'CUT': 
#         if current_date == '12-04-2024': 
#             df.at[idx, 'SCORE'] = (int(row['R1']) - 72) + (int(row['R2']) - 72) 
#         elif current_date == '13-04-2024': 
#             df.at[idx, 'SCORE'] = (int(row['R1']) - 72) + (int(row['R2']) - 72) + max_today
#         elif current_date == '14-04-2024': 
#             df.at[idx, 'SCORE'] = (int(row['R1']) - 72) + (int(row['R2']) - 72) + (int(row['R3']) - 72) + max_today
    

In [67]:
df

,,POS,,PLAYER,SCORE,TODAY,THRU,R1,R2,R3,R4,TOT
0,,T1,-,Max Homa,-6,-1,2:45 PM,67,71,0.0,0.0,138
1,,T1,-,Scottie Scheffler,-6,E,2:35 PM,66,72,0.0,0.0,138
2,,T1,-,Bryson DeChambeau,-6,+1,2:45 PM,65,73,0.0,0.0,138
3,,4,-,Nicolai Højgaard,-4,+1,2:35 PM,67,73,0.0,0.0,140
4,,T5,-,Collin Morikawa,-3,-2,2:25 PM,71,70,0.0,0.0,141
...,...,...,...,...,...,...,...,...,...,...,...,...
85,,-,-,Dustin Johnson,20,-,CUT,78,79,0.0,0.0,157
86,,-,-,Gary Woodland,20,-,CUT,76,81,0.0,0.0,157
87,,-,-,Adrian Meronk,21,-,CUT,78,80,0.0,0.0,158
88,,-,-,Nick Taylor,21,-,CUT,77,81,0.0,0.0,158


In [55]:
filtered_df['TODAY']

0     -1
1      0
2     +1
3     +1
4     -2
      ..
85     -
86     -
87     -
88     -
89     -
Name: TODAY, Length: 89, dtype: object

In [49]:

current_date

'12-04-2024'

In [30]:
scores.columns

Index(['golfer_name', 'score'], dtype='object')

In [24]:
scores.columns.get_level_values(0)

Index(['golfer_name', 'score'], dtype='object')